In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
# Start a SparkSession
import findspark
findspark.init()

In [4]:
# Needed for Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-02-02 20:42:00--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.43MB/s    in 0.6s    

2020-02-02 20:42:02 (1.43 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [0]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataHW").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [0]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://petgis1776-sample-bucket.s3.us-east-2.amazonaws.com/user_data.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("user_data.csv"), sep=",", header=True)

In [7]:
# Show DataFrame
user_data_df.show()

+---+----------+---------+-----------+--------------------+--------------------+------------+
| id|first_name|last_name|active_user|      street_address|               state|    username|
+---+----------+---------+-----------+--------------------+--------------------+------------+
|  1|    Cletus|  Lithcow|      FALSE| 78309 Riverside Way|            Virginia|   ibearham0|
|  2|       Caz|   Felgat|      FALSE| 83 Hazelcrest Place|             Alabama|    wwaller1|
|  3|     Kerri|  Crowson|      FALSE|      112 Eliot Pass|      North Carolina|   ichesnut2|
|  4|   Freddie|    Caghy|      FALSE|     15 Merchant Way|            New York|     tsnarr3|
|  5|   Sadella|    Deuss|      FALSE|    079 Acker Avenue|           Tennessee|   fwherrit4|
|  6|    Fraser|  Korneev|       TRUE|  76084 Novick Court|           Minnesota|  fstappard5|
|  7|    Demott|   Rapson|       TRUE|    86320 Dahle Park|District of Columbia|  lhambling6|
|  8|    Robert|    Poile|      FALSE|1540 Manitowish Hill| 

In [8]:
url ="https://petgis1776-sample-bucket.s3.us-east-2.amazonaws.com/user_payment.csv"
spark.sparkContext.addFile(url)
user_payment_df = spark.read.csv(SparkFiles.get("user_payment.csv"), sep=",", header=True)

# Show DataFrame
user_payment_df.show()

+----------+------------+--------------------+
|billing_id|    username|        cc_encrypted|
+----------+------------+--------------------+
|         1|   ibearham0|a799fcafe47d7fb19...|
|         2|    wwaller1|a799fcafe47d7fb19...|
|         3|   ichesnut2|a799fcafe47d7fb19...|
|         4|     tsnarr3|a799fcafe47d7fb19...|
|         5|   fwherrit4|a799fcafe47d7fb19...|
|         6|  fstappard5|a799fcafe47d7fb19...|
|         7|  lhambling6|a799fcafe47d7fb19...|
|         8|      drude7|a799fcafe47d7fb19...|
|         9|   bspawton8|a799fcafe47d7fb19...|
|        10| rmackeller9|a799fcafe47d7fb19...|
|        11| cdennerleya|a799fcafe47d7fb19...|
|        12|    gsarfasb|a799fcafe47d7fb19...|
|        13| mpichefordc|a799fcafe47d7fb19...|
|        14|     bingryd|a799fcafe47d7fb19...|
|        15|   wheinerte|a799fcafe47d7fb19...|
|        16|    mdrewetf|a799fcafe47d7fb19...|
|        17|droughsedgeg|a799fcafe47d7fb19...|
|        18|     abaakeh|a799fcafe47d7fb19...|
|        19| 

In [9]:
# Join the two DataFrames
joined_df = user_data_df.join(user_payment_df, on="username", how="inner")
joined_df.show()

+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|    username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   ibearham0|  1|    Cletus|  Lithcow|      FALSE| 78309 Riverside Way|            Virginia|         1|a799fcafe47d7fb19...|
|    wwaller1|  2|       Caz|   Felgat|      FALSE| 83 Hazelcrest Place|             Alabama|         2|a799fcafe47d7fb19...|
|   ichesnut2|  3|     Kerri|  Crowson|      FALSE|      112 Eliot Pass|      North Carolina|         3|a799fcafe47d7fb19...|
|     tsnarr3|  4|   Freddie|    Caghy|      FALSE|     15 Merchant Way|            New York|         4|a799fcafe47d7fb19...|
|   fwherrit4|  5|   Sadella|    Deuss|      FALSE|    079 Acker Avenue|           Tennessee|         5|a799fcafe47d7f

In [10]:
# Drop null values
dropna_df = joined_df.dropna()
dropna_df.show()

+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|    username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   ibearham0|  1|    Cletus|  Lithcow|      FALSE| 78309 Riverside Way|            Virginia|         1|a799fcafe47d7fb19...|
|    wwaller1|  2|       Caz|   Felgat|      FALSE| 83 Hazelcrest Place|             Alabama|         2|a799fcafe47d7fb19...|
|   ichesnut2|  3|     Kerri|  Crowson|      FALSE|      112 Eliot Pass|      North Carolina|         3|a799fcafe47d7fb19...|
|     tsnarr3|  4|   Freddie|    Caghy|      FALSE|     15 Merchant Way|            New York|         4|a799fcafe47d7fb19...|
|   fwherrit4|  5|   Sadella|    Deuss|      FALSE|    079 Acker Avenue|           Tennessee|         5|a799fcafe47d7f

In [0]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

# Load in sql function to use columns
from pyspark.sql.functions import col

In [12]:
# Filter for only columns with active users
cleaned_df = dropna_df.filter(col("active_user")==True)
cleaned_df.show()

+-------------+---+----------+-----------+-----------+--------------------+--------------------+----------+--------------------+
|     username| id|first_name|  last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+-------------+---+----------+-----------+-----------+--------------------+--------------------+----------+--------------------+
|   fstappard5|  6|    Fraser|    Korneev|       TRUE|  76084 Novick Court|           Minnesota|         6|a799fcafe47d7fb19...|
|   lhambling6|  7|    Demott|     Rapson|       TRUE|    86320 Dahle Park|District of Columbia|         7|a799fcafe47d7fb19...|
|    wheinerte| 15|   Sadella|      Jaram|       TRUE|7528 Waxwing Terrace|         Connecticut|        15|a799fcafe47d7fb19...|
| droughsedgeg| 17|    Hewitt|    Trammel|       TRUE|    2455 Corry Alley|      North Carolina|        17|a799fcafe47d7fb19...|
|    ydudeniei| 19|       Ted|    Knowlys|       TRUE|      31 South Drive|                Ohio| 

In [13]:
# Create user dataframe to match active_user table
clean_user_df = cleaned_df.select(["id", "first_name", "last_name", "username"]).withColumn("id", cleaned_df["id"].cast(IntegerType()))
clean_user_df.show()

+---+----------+-----------+-------------+
| id|first_name|  last_name|     username|
+---+----------+-----------+-------------+
|  6|    Fraser|    Korneev|   fstappard5|
|  7|    Demott|     Rapson|   lhambling6|
| 15|   Sadella|      Jaram|    wheinerte|
| 17|    Hewitt|    Trammel| droughsedgeg|
| 19|       Ted|    Knowlys|    ydudeniei|
| 23|  Annmarie|     Lafond|     fmyttonm|
| 28|      Toma|     Sokell|   bfletcherr|
| 30|       Ram|    Lefever|     gturleyt|
| 31|    Raddie|    Heindle|    calyukinu|
| 33|    Wallie|       Caws| ckleinlererw|
| 34|    Derril|Varfolomeev|  pshanklandx|
| 39|     Kelcy|     Wheway|    enelane12|
| 40|    Dorree|    Rookeby|    sfollet13|
| 41|    Martyn|       Tott|      mtesh14|
| 43|     Cally|      Thody|   tseyfart16|
| 45|       Ted|   Pittaway|   hfarrier18|
| 48|      Fifi|    Lidgley|     nabbie1b|
| 50|    Ashely|     O'Hern|  ystadding1d|
| 53|   Diannne|Osbaldeston|hhallgalley1g|
| 60|     Sonny|     Jeskin|   ageaveny1n|
+---+------

In [14]:
# Create user dataframe to match billing_info table
clean_billing_df = cleaned_df.select(["billing_id", "street_address", "state", "username"]).withColumn("billing_id", cleaned_df["billing_id"].cast(IntegerType()))
clean_billing_df.show()

+----------+--------------------+--------------------+-------------+
|billing_id|      street_address|               state|     username|
+----------+--------------------+--------------------+-------------+
|         6|  76084 Novick Court|           Minnesota|   fstappard5|
|         7|    86320 Dahle Park|District of Columbia|   lhambling6|
|        15|7528 Waxwing Terrace|         Connecticut|    wheinerte|
|        17|    2455 Corry Alley|      North Carolina| droughsedgeg|
|        19|      31 South Drive|                Ohio|    ydudeniei|
|        23|     35 Oriole Place|             Georgia|     fmyttonm|
|        28|39641 Eggendart Hill|            Maryland|   bfletcherr|
|        30|   9969 Laurel Alley|               Texas|     gturleyt|
|        31|   811 Talmadge Road|                Ohio|    calyukinu|
|        33|   9999 Kenwood Pass|              Oregon| ckleinlererw|
|        34|     4 Jenifer Court|             Florida|  pshanklandx|
|        39|93207 Morningstar...| 

In [15]:
# Create user dataframe to match payment_info table
clean_payment_df = cleaned_df.select(["billing_id", "cc_encrypted"]).withColumn("billing_id", cleaned_df["billing_id"].cast(IntegerType()))
clean_payment_df.show()

+----------+--------------------+
|billing_id|        cc_encrypted|
+----------+--------------------+
|         6|a799fcafe47d7fb19...|
|         7|a799fcafe47d7fb19...|
|        15|a799fcafe47d7fb19...|
|        17|a799fcafe47d7fb19...|
|        19|a799fcafe47d7fb19...|
|        23|a799fcafe47d7fb19...|
|        28|a799fcafe47d7fb19...|
|        30|a799fcafe47d7fb19...|
|        31|a799fcafe47d7fb19...|
|        33|a799fcafe47d7fb19...|
|        34|a799fcafe47d7fb19...|
|        39|a799fcafe47d7fb19...|
|        40|a799fcafe47d7fb19...|
|        41|a799fcafe47d7fb19...|
|        43|a799fcafe47d7fb19...|
|        45|a799fcafe47d7fb19...|
|        48|a799fcafe47d7fb19...|
|        50|a799fcafe47d7fb19...|
|        53|a799fcafe47d7fb19...|
|        60|a799fcafe47d7fb19...|
+----------+--------------------+
only showing top 20 rows



In [16]:
clean_user_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- username: string (nullable = true)



In [0]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cry9bvgbj5ns.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": "x?7svF2JEu5SE7rs",
          "driver":"org.postgresql.Driver"}

In [0]:
# Write DataFrame to active_user table in RDS
clean_user_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)

In [0]:
# Write dataframe to billing_info table in RDS
clean_billing_df.write.jdbc(url=jdbc_url, table='billing_info', mode=mode, properties=config)

In [0]:
# Write dataframe to payment_info table in RDS
clean_payment_df.write.jdbc(url=jdbc_url, table='payment_info', mode=mode, properties=config)